In [1]:
from buildingmotif import BuildingMOTIF
from buildingmotif.model_builder import TemplateBuilderContext
from buildingmotif.dataclasses import Library, Model
from buildingmotif.namespaces import bind_prefixes
from rdflib import Namespace

# setup our buildingmotif instance
bm = BuildingMOTIF("sqlite://", shacl_engine="topquadrant")

In [2]:
# create the model w/ a namespace
BLDG = Namespace("urn:nrel_example/")
bldg = Model.create(BLDG)
bind_prefixes(bldg.graph)
bldg.graph.bind("bldg", BLDG)

s223 = Library.load(ontology_graph="../libraries/223p.ttl")
s223 = Library.load(ontology_graph="../libraries/water.ttl")
nrel = Library.load(directory="../libraries/nrel-223p-templates")
water = Library.load(directory="../libraries/templates")

2024-12-31 12:22:05,744 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/extensions/settings (No row was found when one was required). Check this is loaded into BuildingMOTIF
2024-12-31 12:22:05,745 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/model/core (No row was found when one was required). Check this is loaded into BuildingMOTIF
2024-12-31 12:22:05,746 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/model/equipment (No row was found when one was required). Check this is loaded into BuildingMOTIF
2024-12-31 12:22:05,746 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/vocab/enumeration (No row was found when one was required). Check this is loaded into BuildingMOTIF
2024-12-31 12:22:05,747 | root |  WARNING: An ontology could not resolve a dependency on http:/

In [3]:
# the context helps us build up our model
ctx = TemplateBuilderContext(BLDG)
ctx.add_templates_from_library(nrel)
ctx.add_templates_from_library(water)

## Modeling a Tank with a Flow Sensor

Looking at our templates, we can see a [Tank template](https://datadrivencps.github.io/water-ontology/libraries/templates/tank.html) and a [Sensor template](https://datadrivencps.github.io/water-ontology/libraries/nrel-223p-templates/sensor.html). We will also use the [water flow](https://datadrivencps.github.io/water-ontology/libraries/nrel-223p-templates/water-flow.html) property template to model the actual property being measured by the sensor.

The tank has a few required parameters. It needs a name (like all entities), and either an inlet/outlet connection point pair OR a single bi-directional connection point.

A sensor  needs a name as well as a property it is observing, and a location where that property is observed.

In [4]:
# start with the tank. This instantiates the template with 'my_tank' as the name
# You can assign other parameters here as well. If the value you give to a parameter
# is a string (e.g. 'my_tank') then the builder will automatically make a URL with the BLDG (see cell above) as the namespace
my_tank = ctx['tank'](name='my_tank')

# now create an instance of the water flow property
flow_prop = ctx['water-flow'](name='tank_outlet_water_flow')
flow_prop['name-owner'] = my_tank['out']

In [5]:
# now create the sensor
flow_sensor = ctx['sensor'](name='my_flow_sensor')
flow_sensor['where'] = my_tank['out'] # the sensor measures at the outlet of the tank
flow_sensor['property'] = flow_prop['name'] # associate the sensor's "property" param with the flow property

In [6]:
bldg.add_graph(ctx.compile())
graph = bldg.compile([s223.get_shape_collection()])

In [7]:
# this is the serialization of the model for machine-to-machine
print(graph.serialize(format="turtle"))

@prefix ns1: <http://data.ashrae.org/standard223#> .
@prefix ns2: <http://qudt.org/schema/qudt/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<urn:nrel_example/> a owl:Ontology .

<urn:nrel_example/my_flow_sensor> a ns1:Sensor ;
    rdfs:label "Sensor" ;
    ns1:hasObservationLocation <urn:nrel_example/out_93977d43> ;
    ns1:observes <urn:nrel_example/tank_outlet_water_flow> .

<urn:nrel_example/tank_outlet_water_flow> a ns1:Property,
        ns1:QuantifiableObservableProperty ;
    rdfs:label "Property" ;
    ns1:ofMedium ns1:Fluid-Water ;
    ns2:hasQuantityKind <http://qudt.org/vocab/quantitykind/VolumeFlowRate> ;
    ns2:hasUnit <http://qudt.org/vocab/unit/FT3-PER-MIN> .

<urn:nrel_example/my_tank> a <urn:nawi-water-ontology#Tank> ;
    rdfs:label "Tank" ;
    ns1:cnx <urn:nrel_example/out_93977d43> ;
    ns1:hasConnectionPoint <urn:nrel_example/out_93977d43> ;
    ns1:isConnectionPointOf <urn:nrel_example/out_93977d43>